<a href="https://colab.research.google.com/github/amalsalilan/B3-Developing-Named-Entity-Recognition-NER-Models-for-Financial-Data-Extraction-/blob/main/Custom_with_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U spacy -q

In [2]:
!python -m spacy info



============================== Info about spaCy ==============================

spaCy version    3.8.7                         
Location         /usr/local/lib/python3.12/dist-packages/spacy
Platform         Linux-6.6.105+-x86_64-with-glibc2.35
Python version   3.12.12                       
Pipelines        en_core_web_sm (3.8.0)        



In [3]:
from google.colab import files
uploaded = files.upload()


Saving annotations.json to annotations.json


In [5]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()


In [6]:
import json
f = open('annotations.json')
TRAIN_DATA = json.load(f)

In [7]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./annotations.spacy")

100%|██████████| 9/9 [00:00<00:00, 1188.30it/s]


In [8]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
! python -m spacy train config.cfg --output ./ --paths.train ./annotations.spacy --paths.dev ./annotations.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     20.70    0.00    0.00    0.00    0.00
 42     200        357.73   1683.68  100.00  100.00  100.00    1.00
 95     400          0.00      0.00  100.00  100.00  100.00    1.00
162     600          0.00      0.00  100.00  100.00  100.00    1.00
254     800          0.00      0.00  100.00  100.00  100.00    1.00
354    1000          0.00      0.00  100.00  100.00  100.00    1.00
458    1200          0.00      0.00  100.00  100.00  100.00    1.00
658    1400          0.00      0.00  100.00  100.00  100.00    1.00
858    1600          0.00      0.00 

In [10]:
nlp_ner = spacy.load("/content/model-best")

In [11]:
doc = nlp_ner('''The global crypto market capitalisation tumbled 3.60 percent over the last 24 hours to $2.18 trillion while the total trading volume fell 6.79 percent to $95.77 billion. While DeFi ($15.16 billion) accounted for 15.83 percent of the trading volume, stablecoins ($75.16 billion) made up 78.48 percent. The market dominance of Bitcoin rose 0.29 percent to 40.45 percent today morning. Bitcoin is currently trading at $46,560.09. As for major cryptocurrencies, Bitcoin tumbled 2.46 percent to trade at Rs 37,49,173 while Ethereum fell 4.48 percent at Rs 2,93,527.4. Cardano declined 7.25 percent to Rs 105.4. Avalanche fell 8.77 percent to Rs 8,048, Polkadot tumbled 7.53 percent at Rs 2,137.04 and Litecoin dipped 1.04 percent to Rs 11,725.33 over the last 24 hours. Tether rose 0.12 percent to trade at Rs 80.18. Memecoin SHIB fell 5.61 percent while Dogecoin decreased 4.29 percent to trade at Rs 13.56. LUNA fell around 6.39 percent to Rs 6,579.55. Ethereum-based metaverse game The Sandbox co-founder and operations chief Sebastien Borget viewed the metaverse as a digital nation, saying that it’s validating to see his team’s idea for a community-owned, user-customisable online game world embraced by more people. RELATED STORIES 2021: A Year Of NFT Craze. Elon Musk reveals who he thinks the mysterious Satoshi Nakamoto is. Crypto Learn: How to read Crypto Charts? “Every day, the map is different. There are new landowners and new communities that come and decide to build things next to each other. I feel like it's a digital nation—living and breathing. That's why it's exciting. It's culturally rich, it’s global, and it’s accessible," he continued. Per blockchain data site Glassnode, there are now 71,364,788 addresses holding some amount of Ethereum, the second-largest cryptocurrency by market capitalisation. Meanwhile, a white hat hacker won a bug bounty. In early December, the person known as Leon Spacewalker on Twitter had helped Polygon avert a multibillion-dollar disaster. He reported an exploit in a critical Polygon smart contract that held more than nine billion MATIC tokens on Dec. 3, then worth around $20.2 billion. Core developers rushed a fix by Dec. 5. Spacewalker reportedly won a $2.2 million bug bounty, the blockchain network announced yesterday.''')


In [12]:
spacy.displacy.render(doc, style="ent", jupyter=True)